In [1]:
from vina import Vina
import subprocess

In [3]:
subprocess.run(["./ADFR_prep_receptor.sh -r ../../5EJ5/dG_tests/XTB/int2/6/QM_Active_6/opt.pdb -U '' -o ini_receptor.pdbqt"],shell=True)

Unable to assign MAP type to atom Mg
Sorry, there are no Gasteiger parameters available for atom opt: : MG1113:MG
Sorry, there are no Gasteiger parameters available for atom opt: :WAT1341:O
Sorry, there are no Gasteiger parameters available for atom opt: :WAT1549:O


CompletedProcess(args=["./ADFR_prep_receptor.sh -r ../../5EJ5/dG_tests/XTB/int2/6/QM_Active_6/opt.pdb -U '' -o ini_receptor.pdbqt"], returncode=0)

In [7]:
subprocess.run(["./ADFR_prep_lig.sh -l 6.pdb -U '' -o acc.pdbqt"],shell=True)

CompletedProcess(args=["./ADFR_prep_lig.sh -l 6.pdb -U '' -o acc.pdbqt"], returncode=0)

In [8]:
v = Vina(sf_name='vina')
v.set_receptor('ini_receptor.pdbqt')#loaded receptor
ligname = 'acc.pdbqt'
v.set_ligand_from_file(ligname)
v.compute_vina_maps(center=[-35.284, -35.845, 22.277], box_size=[12,12,12]) 
# Score the current pose
v.dock(exhaustiveness=64, n_poses=10)
writename = 'acc_docked.pdbqt'
v.write_poses(writename, n_poses=10, overwrite=True)

Computing Vina grid ... done.
Performing docking (random seed: -1536121980) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -4.871          0          0
   2       -4.839      1.243      4.237
   3       -4.726       1.06      4.336
   4       -4.713      1.789      4.347
   5       -4.682      1.833       4.34
   6       -4.678      1.698      4.288
   7       -4.631      2.113      3.909
   8       -4.589      1.931      2.707
   9       -4.561      1.359      2.024
  10       -4.524     0.8119      3.883


In [13]:
from openbabel import openbabel

def split_pdbqt_to_pdb_with_acc(input_pdbqt, output_prefix="model"):
    """
    Splits a PDBQT file into separate PDB files for each model using Open Babel
    and labels all residues as 'ACC'.

    Parameters:
    - input_pdbqt: Path to the input PDBQT file.
    - output_prefix: Prefix for the output PDB files.
    """
    obconversion = openbabel.OBConversion()
    obconversion.SetInAndOutFormats("pdbqt", "pdb")

    mol = openbabel.OBMol()
    obconversion.ReadFile(mol, input_pdbqt)  # Read the PDBQT file

    model_count = 0
    while True:
        # Change the residue name to 'ACC'
        for res in openbabel.OBResidueIter(mol):
            res.SetResidueName("ACC")
        
        # Write the current molecule to a new PDB file
        output_filename = f"{output_prefix}_{model_count:03d}.pdb"
        with open(output_filename, "w") as outfile:
            obconversion.WriteFile(mol, output_filename)
        model_count += 1

        # Read the next model
        if not obconversion.Read(mol):
            break

    print(f"Split {model_count} models into separate PDB files with residues labeled as 'ACC'.")

# Example usage:
split_pdbqt_to_pdb("acc_docked.pdbqt")


Split 10 models into separate PDB files.
